# 1. Prepare query data

In [1]:
outputPath = './output.txt'
query = {
    "dataModel": {
        "ProviderName": "",
        "PracticeName": "",
        "PracticeAddress": "",
        "Profession": "MP",
        "Scheme": "EHCVS,VPV13",
        "pageSize": "10",
        "pageActualSize": "10",
        "sortField": "SPName",
        "sortColName": "SPName",
        "sortType": "asc",
        "requestType": "criteria",
        "queryLanguage": "zh-tw",
        "hasResult": "true",
        "IsNonClinic": "AnyClinic",
        "IsFreeSub": ""
        }
    }

districts = ["KC","KT","SSP","WTS","YTM","CW","EAST","SOUTH","WC","ISL","KTS","NORTH","SK","ST","TP","TW","TM","YL"]

# 2. HTML parser

In [2]:
from html.parser import HTMLParser

global divCounter, ok, hasValue, dataArr
dataArr = []
divCounter = 0
ok = False
hasValue = True

class MyHTMLParser(HTMLParser):
    # def __init__(self):
    #     self.divCounter = 0

    def handle_starttag(self, tag, attrs):
        global divCounter, ok
        divCounter += 1
        if divCounter == 3 and tag == 'div':
            ok = True

    def handle_endtag(self, tag):
        global divCounter, ok, hasValue
        divCounter -= 1
        ok = False

    def handle_data(self, data):
        global divCounter, ok, hasValue, dataArr
        if divCounter == 3:
            with open(outputPath, 'a+') as f:
                line = data.strip()
                if len(line) > 0 and line[0] == '{':
                    dataArr.append(line)
                    f.write(line)
                    f.write('\n')
                    hasValue = True


# 3. Start parsing gov hcv site

In [3]:
import requests
import urllib

cookies = { "ASP.NET_SessionId": "paju1jhifihxd0ewvzmpg2fk" }
headers = {
    "Accept": "text/html, */*; q=0.01",
}

for d in districts:
    pageIndex = 1
    dataArr = []
    while True:
        beforeLen = len(dataArr)

        query["dataModel"]["District"] = d
        query["dataModel"]["pageIndex"] = pageIndex

        x = urllib.parse.urlencode((query)).replace('%27', '%22').replace('+', '')
        r = requests.get('https://apps.hcv.gov.hk/Public/tc/SPS/GetResult?%s'%(x), params=query, cookies=cookies, headers=headers)

        parser = MyHTMLParser()
        parser.feed(r.text)
        pageIndex += 1
        afterLen = len(dataArr)
        if beforeLen == afterLen:
            break

# 4. Convet output data to csv format

In [5]:
import json
import csv

with open(outputPath, 'r') as f:
    arr = f.readlines()

    with open('pcv13.csv', 'w', newline='') as csvfile:
        for a in arr:
            d = json.loads(a)
            spamwriter = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow([d['SPName'], d['PracticeName'], d['PracticeAddress'], d['PracticePhoneNo'], d['DistrictName']])
            

# 5. Please delete output.txt once exported